In [12]:
# !pip install nltk
# !pip install accelerate
# !pip install datasets

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

In [2]:
# nltk.download("movie_reviews")
fieldids = movie_reviews.fileids()
reviews = [ movie_reviews.raw(fieldid) for fieldid in fieldids ]
categories = [ movie_reviews.categories(fieldid) for fieldid in fieldids ]

In [22]:
# labeled_categories = LabelEncoder().fit_transform(categories)
labeled_categories = LabelEncoder().fit_transform(categories).tolist()
labeled_categories

c:\ProgramData\miniconda3\envs\study-env2\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [24]:
X_train, X_test ,y_train, y_test = \
    train_test_split(reviews, labeled_categories, stratify=labeled_categories, random_state=42)

In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_inputs = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
test_inputs = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [26]:
print( train_inputs['input_ids'].shape, test_inputs['input_ids'].shape )

for item in train_inputs.items():
    print(item[1][2].shape)
    break

torch.Size([1500, 512]) torch.Size([500, 512])
torch.Size([512])


In [27]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        pass

    def __getitem__(self, idx):
        input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }
        input['label'] = torch.tensor(self.labels[idx])
        return input
    
    def __len__(self):
        return len(self.labels)
    
train_dataset = ReviewDataset(train_inputs, y_train)
test_dataset = ReviewDataset(test_inputs, y_test)

In [28]:
for data in train_dataset:
    print(data.keys())
    # print(data)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])


C:\Users\Administrator.User -2023YNCQT\AppData\Local\Temp\ipykernel_16676\3149890996.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


In [29]:
device_name = "cuda:0" if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
device

device(type='cuda', index=0)

In [30]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

metric = load_metric('accuracy')
def compute_metrics(evaluation_dataset):
    logits, labels = evaluation_dataset
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

train_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
    args=train_args
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\ProgramData\miniconda3\envs\study-env2\lib\site-packages\datasets\load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/940 [00:00<?, ?it/s]

C:\Users\Administrator.User -2023YNCQT\AppData\Local\Temp\ipykernel_16676\3149890996.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


KeyboardInterrupt: 

In [30]:
!pip list | findstr accelerate